In [164]:
from itertools import combinations
import numpy as np
import random

GOAL
Cost minimization

## Simple Test Problem

In [165]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('problems/test_problem.npy')

## Common tests

In [166]:
p_g = np.load('problems/problem_g_100.npy')
p_r1 = np.load('problems/problem_r1_100.npy')
p_r2 = np.load('problems/problem_r2_100.npy')

In [167]:
print(p_g)

[[  0.         179.92038688 176.126132   ... 162.61509016 407.15461186
  141.77638964]
 [179.92038688   0.         278.35892376 ... 116.33000682 281.45211088
  154.17643486]
 [176.126132   278.35892376   0.         ... 178.74068495 375.52755063
  312.44633915]
 ...
 [162.61509016 116.33000682 178.74068495 ...   0.         245.06340681
  224.97614312]
 [407.15461186 281.45211088 375.52755063 ... 245.06340681   0.
  434.61707535]
 [141.77638964 154.17643486 312.44633915 ... 224.97614312 434.61707535
    0.        ]]


In [168]:
print(p_r1)

[[  0.           4.9766      65.54479501 ...   7.83902018  27.41995194
   87.69299895]
 [  9.36499057   0.         110.87932346 ...  19.63789468  30.94641048
   50.11884753]
 [ 35.88566785  54.64069406   0.         ...  66.45444707  87.16585662
   13.71356104]
 ...
 [ 15.7861903   13.58665954  37.36828725 ...   0.          33.44981139
   28.40778893]
 [ 43.48793718  32.44459687  80.22882128 ...  18.97247219   0.
   68.05734486]
 [ 38.95399487  52.21881804  12.94487068 ...  67.75221274  60.95591025
    0.        ]]


In [169]:
print(p_r2)

[[ -9.15367559  47.74344577  40.16586609 ... -40.84436694  49.89304781
   -3.80798436]
 [-49.59100483 -49.17049366 -13.77660833 ...   9.24571587 -10.78074211
  -31.92741522]
 [ 44.58858581  13.16415431  24.0761291  ...  46.47632671 -22.10624941
  -20.34470381]
 ...
 [  1.09354585   9.06381007  16.15032321 ...  32.66745074  41.05556601
   35.1475426 ]
 [ 47.09316012  17.95291657   8.44268749 ...  33.69341475  -3.4474984
  -17.256382  ]
 [ -5.79703165  -5.79169305 -12.96217049 ...  11.97955136  42.27871638
    7.40063403]]


In [170]:
def check_negative_value(problem):
    # Negative values?
    return np.any(problem < 0)


def check_diagonal_all_zero(problem):
    # Diagonal is all zero?
    return np.allclose(np.diag(problem), 0.0)


def check_symmetry(problem):
    # Symmetric matrix?
    return np.allclose(problem, problem.T)


def check_triangular_inequality(problem):
    # Triangular inequality
    return all(
        problem[x, y] <= problem[x, z] + problem[z, y]
        for x, y, z in list(combinations(range(problem.shape[0]), 3))
    )

In [171]:
print(f"check negative value | p_g: {check_negative_value(p_g)}")
print(f"check diagonal all 0s | p_g: {check_diagonal_all_zero(p_g)}")
print(f"check symmetry | p_g: {check_symmetry(p_g)}")
print(f"check triangular inequality | p_g {check_triangular_inequality(p_g)}")
print()
print(f"check negative value | p_r1: {check_negative_value(p_r1)}")
print(f"check diagonal all 0s | p_r1: {check_diagonal_all_zero(p_r1)}")
print(f"check symmetry | p_r1: {check_symmetry(p_r1)}")
print(f"check triangular inequality | p_r1 {check_triangular_inequality(p_r1)}")
print()
print(f"check negative value | p_r2: {check_negative_value(p_r2)}")
print(f"check diagonal all 0s | p_r2: {check_diagonal_all_zero(p_r2)}")
print(f"check symmetry | p_r2: {check_symmetry(p_r2)}")
print(f"check triangular inequality | p_r2 {check_triangular_inequality(p_r2)}")
print()

check negative value | p_g: False
check diagonal all 0s | p_g: True
check symmetry | p_g: True
check triangular inequality | p_g True

check negative value | p_r1: False
check diagonal all 0s | p_r1: True
check symmetry | p_r1: False
check triangular inequality | p_r1 False

check negative value | p_r2: True
check diagonal all 0s | p_r2: False
check symmetry | p_r2: False
check triangular inequality | p_r2 False



In [172]:
# HYPERPARAMETERS

POPULATION_SIZE = 100
NUM_GENERATIONS = 1000
ELITISM_SIZE = 2
TOURNAMENT_K = 3
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.2

In [173]:
def create_initial_population(pop_size, num_cities):
    """Creates a list of random tours (solutions)."""
    population = []
    for _ in range(pop_size):
        tour = np.random.permutation(num_cities)
        population.append(tour)
    return population

In [174]:
def calculate_fitness(tour, distance_matrix):
    """
    Calculates the total cost (fitness) of a single tour.
    This is the value we want to MINIMIZE.
    Works for both symmetric and asymmetric matrices.
    """
    total_cost = 0
    num_cities = len(tour)
    for i in range(num_cities):
        city_a = tour[i]
        # Use modulo (%) to wrap around to the start city
        city_b = tour[(i + 1) % num_cities]
        
        # This correctly takes the cost from A -> B
        total_cost += distance_matrix[city_a, city_b]
        
    return total_cost

In [175]:
def tournament_selection(population, fitness_scores, k):
    """
    Selects one parent using k-tournament selection.    
    Picks k random individuals and returns the fittest one.
    """
    indices = np.random.choice(len(population), k, replace=False)
    k_fitnesses = [fitness_scores[i] for i in indices]
    best_local_idx = np.argmin(k_fitnesses)
    best_global_idx = indices[best_local_idx]
    
    return population[best_global_idx]

In [176]:
def crossover(parent1, parent2):
    """
    Perform crossover
    """
    num_cities = len(parent1)
    offspring = np.full(num_cities, -1)
    
    # Select two random cut points
    start, end = sorted(np.random.choice(num_cities, 2, replace=False))
    
    # Copy the slice from parent1
    offspring[start:end] = parent1[start:end]
    
    # Fill remaining slots from parent2
    parent2_ptr = 0
    offspring_ptr = 0
    cities_in_offspring = set(offspring[start:end])
    
    while -1 in offspring:
        # Find next available slot in offspring
        if offspring_ptr == start:
            offspring_ptr = end
            
        # Get city from parent2
        city_to_add = parent2[parent2_ptr]
        parent2_ptr += 1
        
        if city_to_add not in cities_in_offspring:
            offspring[offspring_ptr] = city_to_add
            offspring_ptr += 1
            
    return offspring

In [177]:
# DIFFERENT KINDS OF MUTATIONS


def swap_mutation(tour):
    """Performs a simple swap mutation on a tour (modifies in-place)."""
    idx1, idx2 = np.random.choice(len(tour), 2, replace=False)
    tour[idx1], tour[idx2] = tour[idx2], tour[idx1]


def inversion_mutation(tour):
    """
    Performs an inversion mutation (modifies in-place).
    """
    # Select two random cut points
    start, end = sorted(np.random.choice(len(tour), 2, replace=False))
    if start >= end:
        return 
    
    # Reverse the sub-sequence in-place
    tour[start:end] = tour[start:end][::-1]

In [178]:
def algo_1(distance_matrix, pop_size, num_generations, elitism_size, k, crossover_rate, mutation_rate):
    """
    Runs the main evolutionary algorithm loop.
    """
    num_cities = distance_matrix.shape[0]
    
    # Initializing
    population = create_initial_population(pop_size, num_cities)
    best_tour_so_far = None
    best_fitness_so_far = float('inf')
    
    fitness_history = []

    for gen in range(num_generations):
        # Calculate fitness for the entire population
        fitness_scores = [calculate_fitness(tour, distance_matrix) for tour in population]
        
        # Find the indices of the N best individuals,
        # The first index is the one that refers to the individual with the best fitness
        elite_indices = np.argsort(fitness_scores)[:elitism_size]
        # creating a new population composed of the best individuals of the previous
        new_population = [population[i].copy() for i in elite_indices]
        
        # Track the best-ever solution
        best_current_idx = elite_indices[0] 
        current_best_fitness = fitness_scores[best_current_idx]
        
        if current_best_fitness < best_fitness_so_far:
            best_fitness_so_far = current_best_fitness
            best_tour_so_far = population[best_current_idx].copy()
            print(f"Gen {gen}: New best cost = {best_fitness_so_far:.2f}")

        # Add the best cost of this generation to the history
        fitness_history.append(current_best_fitness)

        # In this loop we compute the generation
        while len(new_population) < pop_size:
            # Selecting 2 parents using tournament selection
            parent1 = tournament_selection(population, fitness_scores, k)
            parent2 = tournament_selection(population, fitness_scores, k)
            
            # performing the cross over (random decision defined by crossover_rate)
            if random.random() < crossover_rate:
                offspring = crossover(parent1, parent2)
            else:
                offspring = parent1.copy() 
            
            # performing the mutation
            # (again, its a random decision that doesn't even depend on whether the crossover is done or not)
            if random.random() < mutation_rate:
                # Here e we can choose to use inversion_mutation() and swap_mutation()
                inversion_mutation(offspring)
                
            new_population.append(offspring)

        # Replace the old population with the new one
        population = new_population

    return best_tour_so_far, best_fitness_so_far, fitness_history

In [182]:
best_tour_p_g, best_fitness_p_g, _ = algo_1(
        distance_matrix=p_g,
        pop_size=POPULATION_SIZE,
        num_generations=NUM_GENERATIONS,
        elitism_size=ELITISM_SIZE,
        k=TOURNAMENT_K,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE
    )

print(f"Best tour (p_g): {best_tour_p_g}")
print(f"Total fitness (p_g): {best_fitness_p_g:.2f}\n")

Gen 0: New best cost = 23165.07
Gen 1: New best cost = 22552.42
Gen 3: New best cost = 21660.17
Gen 5: New best cost = 20694.36
Gen 7: New best cost = 19786.87
Gen 8: New best cost = 19133.20
Gen 9: New best cost = 19064.78
Gen 11: New best cost = 18530.67
Gen 13: New best cost = 18183.36
Gen 15: New best cost = 18015.96
Gen 16: New best cost = 17292.47
Gen 19: New best cost = 17164.04
Gen 20: New best cost = 17038.04
Gen 22: New best cost = 17017.56
Gen 23: New best cost = 16455.45
Gen 24: New best cost = 16155.04
Gen 26: New best cost = 15965.16
Gen 27: New best cost = 15580.95
Gen 29: New best cost = 15298.05
Gen 31: New best cost = 14807.43
Gen 32: New best cost = 14787.57
Gen 33: New best cost = 14368.63
Gen 36: New best cost = 14295.26
Gen 37: New best cost = 14019.64
Gen 39: New best cost = 13827.88
Gen 42: New best cost = 13683.90
Gen 44: New best cost = 13553.51
Gen 46: New best cost = 13521.93
Gen 48: New best cost = 13364.02
Gen 50: New best cost = 13329.68
Gen 52: New best 

In [183]:
best_tour_p_r1, best_fitness_p_r1, _ = algo_1(
        distance_matrix=p_r1,
        pop_size=POPULATION_SIZE,
        num_generations=NUM_GENERATIONS,
        elitism_size=ELITISM_SIZE,
        k=TOURNAMENT_K,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE
)

print(f"Best tour (p_g): {best_tour_p_r1}")
print(f"Total fitness (p_g): {best_fitness_p_r1:.2f}\n")

Gen 0: New best cost = 4679.62
Gen 1: New best cost = 4455.34
Gen 3: New best cost = 4337.46
Gen 4: New best cost = 4167.48
Gen 5: New best cost = 3938.24
Gen 8: New best cost = 3833.42
Gen 10: New best cost = 3697.80
Gen 11: New best cost = 3657.83
Gen 13: New best cost = 3607.54
Gen 14: New best cost = 3529.62
Gen 15: New best cost = 3508.45
Gen 16: New best cost = 3452.73
Gen 17: New best cost = 3339.01
Gen 20: New best cost = 3202.37
Gen 21: New best cost = 3191.50
Gen 24: New best cost = 3145.64
Gen 26: New best cost = 3142.66
Gen 27: New best cost = 3142.53
Gen 28: New best cost = 3099.35
Gen 29: New best cost = 3096.68
Gen 31: New best cost = 3067.32
Gen 32: New best cost = 3016.31
Gen 33: New best cost = 2983.99
Gen 34: New best cost = 2978.83
Gen 36: New best cost = 2973.20
Gen 38: New best cost = 2962.02
Gen 40: New best cost = 2960.18
Gen 41: New best cost = 2951.12
Gen 42: New best cost = 2949.54
Gen 43: New best cost = 2877.19
Gen 46: New best cost = 2849.68
Gen 48: New be

In [185]:
best_tour_p_r2, best_fitness_p_r2, _ = algo_1(
        distance_matrix=p_r2,
        pop_size=POPULATION_SIZE,
        num_generations=NUM_GENERATIONS,
        elitism_size=ELITISM_SIZE,
        k=TOURNAMENT_K,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE
)

print(f"Best tour (p_g): {best_tour_p_r2}")
print(f"Total fitness (p_g): {best_fitness_p_r2:.2f}\n")

Gen 0: New best cost = -642.95
Gen 1: New best cost = -784.19
Gen 2: New best cost = -1080.12
Gen 3: New best cost = -1089.97
Gen 4: New best cost = -1157.33
Gen 5: New best cost = -1227.22
Gen 6: New best cost = -1358.69
Gen 7: New best cost = -1367.46
Gen 8: New best cost = -1382.53
Gen 9: New best cost = -1418.49
Gen 10: New best cost = -1451.08
Gen 11: New best cost = -1542.13
Gen 13: New best cost = -1648.01
Gen 15: New best cost = -1676.17
Gen 17: New best cost = -1789.78
Gen 18: New best cost = -1822.63
Gen 20: New best cost = -1925.76
Gen 21: New best cost = -1939.57
Gen 22: New best cost = -1949.49
Gen 24: New best cost = -2006.36
Gen 26: New best cost = -2079.64
Gen 29: New best cost = -2164.23
Gen 30: New best cost = -2177.51
Gen 31: New best cost = -2272.39
Gen 35: New best cost = -2443.64
Gen 38: New best cost = -2507.75
Gen 39: New best cost = -2622.66
Gen 42: New best cost = -2659.09
Gen 44: New best cost = -2672.97
Gen 45: New best cost = -2696.95
Gen 46: New best cost 